# Year 2023 Day 21

In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray  # pyright: ignore[reportUnusedImport,reportMissingTypeStubs]
from pathlib import Path
from advent_of_code.common.common import get_puzzle_input_file_path, numpy_2d_to_xarray_row_col
from advent_of_code.y_2023.problem_202321 import (
    parse_text_input,
)
from advent_of_code.y_2023.problem_202321 import (
    get_starting_position,
    run_steps_details,
    create_diamond_int_array,
    create_diamond_mask_array,
    get_free_cells_xda,
    evaluate_quadratic_equation,
    compute_points_for_interpolation,
)

## With Mock Input

In [ ]:
full_size = True 
side = 131 if full_size else 11
middle = 65 if full_size else 5 

arr = np.zeros((side, side), dtype=np.bool_)
assert side // 2 == middle
starting_pos  = middle
arr[middle][middle] # starting pos

In [ ]:
diamond_array = create_diamond_int_array(numpy_2d_to_xarray_row_col(arr))
diamond_array

In [ ]:
diamond_array[middle]

In [ ]:
diamond_mask = create_diamond_mask_array(diamond_array)
diamond_mask.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
top_left = diamond_mask[:middle, :middle]
top_left

In [ ]:
top_left = diamond_mask[:middle+2, :middle+2]
top_left

## With Actual Input

In [ ]:
text = (get_puzzle_input_file_path(2023, 21)).read_text()
garden = parse_text_input(text)
garden.coords

In [ ]:
free = get_free_cells_xda(garden)

In [ ]:
free.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
(~free).hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
create_diamond_mask_array(free).hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

### Observation on Data

- All edges are free
- The middle row and middle cols are free
- Presence of a diagonal path

Hence the image can be safely divided into 8 half-quadrants

In [ ]:
free.coords

In [ ]:
assert free.shape == (side, side)
assert (free[0]).all().item()
assert (free[-1]).all().item()
assert (free[:, 0]).all().item()
assert (free[:, -1]).all().item()
assert free[middle].all().item()
assert free[:, middle].all().item()

In [ ]:
initial_pos = get_starting_position(garden)
max_iter = 65 * 2

history, reached, reached_even_xda, reached_odd_xda = run_steps_details(garden, initial_pos, max_iter)

In [ ]:
reached_odd_xda.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
reached_even_xda.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

### Check part 1 can be found again

In [ ]:
max_iter = 64 # we will want the even
_, _, reached_even_xda_p1, reached_odd_xda_p1 = run_steps_details(
    garden, initial_pos, max_iter
)

In [ ]:
diamond_1x1 = create_diamond_mask_array(reached_even_xda_p1)
diamond_1x1.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
assert reached_even_xda_p1.where(diamond_1x1, 0).sum() == 3740

### 1x1

In [ ]:
odd = reached_odd_xda
eve = reached_even_xda

In [ ]:
odd.where(create_diamond_mask_array(odd), 0).sum().item()

In [ ]:
assert eve.where(create_diamond_mask_array(eve), 0).sum().item() == 3740 
# Part 1 also works when using the eve diagram (as 65 = 64 + 1, we juste iterated one more time)

In [ ]:
concat_1 = odd
diamond_1x1 = create_diamond_mask_array(concat_1)

### 3x3 concat

Note: The step count is **EVEN**, so the even array must be placed back at the center of the mosaic.
This cellular automata kind of "flips" every turn.


In [ ]:
assert middle + 1 * odd.row.size == 196  # 65 + 1 * 131

In [ ]:
new_coords3 = list(range(3 * odd.row.size))
concat_3 = xr.concat(
    [
        xr.concat([eve, odd, eve], dim="col"),
        xr.concat([odd, eve, odd], dim="col"),
        xr.concat([eve, odd, eve], dim="col"),
    ],
    dim="row",
).assign_coords(dict(row=new_coords3, col=new_coords3))
concat_3

In [ ]:
concat_3.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
diamond_3x3 = create_diamond_mask_array(concat_3)
diamond_3x3.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
concat_3.where(diamond_3x3, 0).sum().item()

In [ ]:
concat_3

### 5x5 concat

Note: The step count is odd, so odd must be placed back at the center of the mosaic

In [ ]:
assert middle + 2 * odd.row.size == 327 # 65 + 2 * 131

In [ ]:
new_coords5 = list(range(5 * odd.row.size))
concat_5 = xr.concat(
    [
        xr.concat([odd, eve, odd, eve, odd], dim="col"),
        xr.concat([eve, odd, eve, odd, eve], dim="col"),
        xr.concat([odd, eve, odd, eve, odd], dim="col"),
        xr.concat([eve, odd, eve, odd, eve], dim="col"),
        xr.concat([odd, eve, odd, eve, odd], dim="col"),
    ],
    dim="row",
).assign_coords(dict(row=new_coords5, col=new_coords5))
concat_5

In [ ]:
concat_5.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
diamond_5x5 = create_diamond_mask_array(concat_5)
diamond_5x5.hvplot.image(y="row", x="col", data_aspect=1, cmap="viridis")

In [ ]:
concat_5.where(diamond_5x5, 0).sum().item()

### Interpolation

In [ ]:
x, y = compute_points_for_interpolation(
    concat_1, diamond_1x1, concat_3, diamond_3x3, concat_5, diamond_5x5
)

In [ ]:
assert evaluate_quadratic_equation(x, y, 0) == y[0]
assert evaluate_quadratic_equation(x, y, 1) == y[1]
assert evaluate_quadratic_equation(x, y, 2) == y[2]

In [ ]:
result = evaluate_quadratic_equation(x, y, 26501365 // 131)

In [ ]:
# Too high
# The error was because I forgot to put the even array at the center of the 3x3 mosaic
# Each macro step changes the parity
assert result < 635572702833258
result